<a href="https://qworld.net" target="_blank" align="left"><img src="../qworld/images/header.jpg"  align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b><font color="blue"> Solutions for </font>Grover's Search: Implementation </b></font>

_prepared by Maksim Dimitrijev and Özlem Salehi_

_ProjectQ adaptation by Vishal Sharathchandra Bajpe and Marija Šćekić_
<br><br>

<a id="task2"></a>
## Task 2


Let $N=4$. Implement the query phase and check the unitary matrix for the query operator. Note that we are interested in the top-left $4 \times 4$ part of the matrix since the remaining parts are due to the ancilla qubit.

You are given a function $f$ and its corresponding quantum operator $U_f$. First run the following cell to load operator $U_f$. Then you can make queries to $f$ by applying the operator $U_f$ via the following command:

<pre>Uf(qengine,qubit).

In [ ]:
%run quantum.py

Now use phase kickback to flip the sign of the marked element:

<ul>
    <li>Set output qubit (qreg[2]) to $\ket{-}$ by applying X and H.</li>
    <li>Apply operator $U_f$
    <li>Set output qubit (qreg[2]) back.</li>
</ul>

(Can you guess the marked element by looking at the unitary matrix?)

<h3>Solution</h3>

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, UnitarySimulator
from projectq.setups.default import get_engine_list


qdrawer = CircuitDrawerMatplotlib()
qengine = MainEngine(backend = UnitarySimulator(), engine_list = [qdrawer]+get_engine_list())

qubits = qengine.allocate_qureg(3)

# Apply X gate
X | qubits[2]
H | qubits[2]

Barrier | qubits

# Call oracle
Uf(qengine,qubits)

Barrier | qubits

# Put ancilla back into state |0>
H | qubits[2]
X | qubits[2]

qengine.flush()

circuit_unitary = qengine.backend.unitary


# We are interested in the top-left 4x4 part
for i in range(4):
    s=""
    for j in range(4):
        val = str(round(circuit_unitary[i][j].real,3))
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)
    
    
qdrawer.draw()

All(Measure) | qubits # Just to remove warnings

<a id="task3"></a>
## Task 3


Let $N=4$. Implement the inversion operator and check whether you obtain the following matrix:

$\mymatrix{cccc}{-0.5 & 0.5 & 0.5 & 0.5 \\ 0.5 & -0.5 & 0.5 & 0.5 \\ 0.5 & 0.5 & -0.5 & 0.5 \\ 0.5 & 0.5 & 0.5 & -0.5}$.

<h3>Solution</h3>

In [ ]:
def inversion(qengine,qubits):
    
    # Step 1
    H | qubits[1]
    H | qubits[0]
    
    # Step 2
    X | qubits[1]
    X | qubits[0]
    
    # Step 3
    C(NOT,2) | (qubits[1],qubits[0],qubits[2])
    
    # Step 4
    X | qubits[1]
    X | qubits[0]
    
    # Step 5
    X | qubits[2]
    
    # Step 6
    H | qubits[1]
    H | qubits[0]
    

Below you can check the matrix of your inversion operator and how the circuit looks like. We are interested in top-left $4 \times 4$ part of the matrix, the remaining parts are because we used ancilla qubit.

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, UnitarySimulator
from projectq.setups.default import get_engine_list

qdrawer = CircuitDrawerMatplotlib()
qengine = MainEngine(backend = UnitarySimulator(), engine_list = [qdrawer]+get_engine_list())

qubits = qengine.allocate_qureg(3)

# Set ancilla
X | qubits[2]
H | qubits[2]

Barrier | qubits

# Call inversion function
inversion(qengine,qubits)

Barrier | qubits

# Put ancilla back into state |0>
H | qubits[2]
X | qubits[2]

qengine.flush()

circuit_unitary = qengine.backend.unitary

# We are interested in the top-left 4x4 part
for i in range(4):
    s=""
    for j in range(4):
        val = str(round(circuit_unitary[i][j].real,3))
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)

qdrawer.draw()
All(Measure) | qubits # Just to remove warnings

<a id="task4"></a>
## Task 4: Testing Grover's search

Now we are ready to test our operations and run Grover's search. Suppose that there are 4 elements in the list and try to find the marked element.

You are given the operator $U_f$. First run the following cell to load it. You can access it via <pre>Uf(qengine,qubit).</pre>
qubits[2] is the ancilla qubit and it is shared by the query and the inversion operators.
    
Which state do you observe the most? 

In [ ]:
%run quantum.py

<h3>Solution</h3>

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, Simulator
from projectq.setups.default import get_engine_list

outputs = {}

for i in range(1000):
    qdrawer = CircuitDrawerMatplotlib()
    qengine = MainEngine(backend = Simulator(), engine_list = [qdrawer]+get_engine_list())

    qubits = qengine.allocate_qureg(3)

    # Grover

    # Initial step - equal superposition
    All(H) | qubits

    # Set Ancilla
    X | qubits[2]
    H | qubits[2]

    Barrier | qubits

    # Change the number of iterations
    iterations = 1

    # Grover's iterations
    for i in range(iterations):
        # Query
        Uf(qengine,qubits)
        Barrier | qubits

        # Inversion
        inversion(qengine,qubits)
        Barrier | qubits

    # Put ancilla back into state |0>
    H | qubits[2]
    X | qubits[2]
    
    All(Measure) | qubits
    
    output = ""
    for q in qubits:
        output = output + str(int(q))
    if output in outputs: outputs[output] += 1
    else: outputs[output] = 1

print(outputs)
print()
for output in outputs:
    print(output,"is observed",outputs[output],"times")


qdrawer.draw()
All(Measure) | qubits

<a id="task5"></a>
## Task 5 (Optional, challenging)

Implement the inversion operation for $n=3$ ($N=8$). This time you will need 5 qubits - 3 for the operation, 1 for ancilla, and one more qubit to implement not gate controlled by three qubits.

In the implementation the ancilla qubit will be qubit 3, while qubits for control are 0, 1 and 2; qubit 4 is used for the multiple control operation. As a result you should obtain the following values in the top-left $8 \times 8$ entries:

$\mymatrix{cccccccc}{-0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75}$.

<h3>Solution</h3>

In [ ]:
def big_inversion(qengine,qubit):
    
    for i in range(3):
        H | qubit[i]
        X | qubit[i]
        
    Barrier | qubit
    
    C(NOT,2) | (qubit[1],qubit[0],qubit[4])
    C(NOT,2) | (qubit[2],qubit[4],qubit[3])
    C(NOT,2) | (qubit[1],qubit[0],qubit[4])
    
    Barrier | qubit
    
    for i in range(3):
        X | qubit[i]
        H | qubit[i]
        
    X | qubit[3]

Below you can check the matrix of your inversion operator. We are interested in the top-left $8 \times 8$ part of the matrix, the remaining parts are because of additional qubits.

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, UnitarySimulator
from projectq.setups.default import get_engine_list
measurement_dict = {}

qdrawer = CircuitDrawerMatplotlib()
qengine = MainEngine(backend = UnitarySimulator(), engine_list = [qdrawer]+get_engine_list())

qubit = qengine.allocate_qureg(5)

# Set Ancilla
X | qubit[3]
H | qubit[3]

# Apply Inversion
big_inversion(qengine,qubit)

# Put ancilla back into state |0>
H | qubit[3]
X | qubit[3]

Barrier | qubit

qengine.flush()
All(Measure)| qubit

circuit_unitary= qengine.backend.unitary

# We are interested in the top-left 8x8 part
for i in range(8):
    s=""
    for j in range(8):
        val = str(round(circuit_unitary[i][j].real,3))
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)

qdrawer.draw() 
All(Measure)| qubit

<a id="task6"></a>
## Task 6: Testing Grover's search for 8 elements (Optional, challenging)

Now we will test Grover's search on 8 elements.

You are given the operator $U_{f_8}$. First run the following cell to load it. You can access it via:

<pre>Uf_8(circuit,qreg)</pre>
    
Which state do you observe the most?

In [ ]:
%run quantum.py

<h3>Solution</h3>

In [ ]:
def big_inversion(qengine,qubit):
    
    for i in range(3):
        H | qubit[i]
        X | qubit[i]
    
    C(NOT,2) | (qubit[1],qubit[0],qubit[4])
    C(NOT,2) | (qubit[2],qubit[4],qubit[3])
    C(NOT,2) | (qubit[1],qubit[0],qubit[4])
    
    for i in range(3):
        X | qubit[i]
        H | qubit[i]
        
    X | qubit[3]

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, Simulator
from projectq.setups.default import get_engine_list

outputs = {}

for i in range(1000):
    qdrawer = CircuitDrawerMatplotlib()
    qengine = MainEngine(backend = Simulator(), engine_list = [qdrawer]+get_engine_list())

    qubit = qengine.allocate_qureg(5)
    
    # Set Ancilla
    X | qubit[3]
    H | qubit[3]
    
    for i in range(3):
        H | qubit[i]
    
    Barrier | qubit
    
    #Try 1,2,6,12 8iterations of Grover
    for i in range(2):
        Uf_8(qengine,qubit)
        Barrier | qubit
        big_inversion(qengine,qubit)
        Barrier | qubit
    
    # Put ancilla back into state |0>
    H | qubit[3]
    X | qubit[3]
    
    Barrier | qubit
    
    All(Measure) | qubit
    
    output = ""
    for i in range(3): # qubits 0,1,2 - skip qubits 3,4
        q = qubit[i]
        output = output + str(int(q))
    if output in outputs: outputs[output] += 1
    else: outputs[output] = 1
    
print(outputs)
print()
for output in outputs:
    print(output,"is observed",outputs[output],"times")
    
qdrawer.draw()
All(Measure) | qubit

<a id="task8"></a>
## Task 8

Implement an oracle function which marks the element 00. Run Grover's search with the oracle you have implemented. 

<h3>Solution</h3>

In [ ]:
def oracle_00(qengine,qubit):
    X | qubit[0]
    X | qubit[1]
    C(NOT,2) | (qubit[0],qubit[1],qubit[2])
    X | qubit[0]
    X | qubit[1]

In [ ]:
# Import all required objects and methods
from projectq import MainEngine
from projectq.ops import H,X,Measure,All, C, NOT, Barrier
from projectq.backends import CircuitDrawerMatplotlib, Simulator
from projectq.setups.default import get_engine_list

outputs = {}

for i in range(100):
    qdrawer = CircuitDrawerMatplotlib()
    qengine = MainEngine(backend = Simulator(), engine_list = [qdrawer]+get_engine_list())

    qubit = qengine.allocate_qureg(3)
    
    for i in range(2):
        H | qubit[i]
        
    # Set Ancilla
    X | qubit[2]
    H | qubit[2]

    Barrier | qubit
    
    #Try 1,2,6,12 8iterations of Grover
    iterations=1
    
    for i in range(iterations):
        oracle_00(qengine,qubit)
        Barrier | qubit
        inversion(qengine,qubit)
        Barrier | qubit
    
    # Put ancilla back into state |0>
    H | qubit[2]
    X | qubit[2]
    
    Barrier | qubit
    
    All(Measure) | qubit
        
    output = ""
    for i in range(2): # qubits 0,1 - skip qubit 2
        q = qubit[i]
        output = output + str(int(q))
    if output in outputs: outputs[output] += 1
    else: outputs[output] = 1
    
print(outputs)
print()
for output in outputs:
    print(output,"is observed",outputs[output],"times")
    
qdrawer.draw()
All(Measure) | qubit